In [1]:
class Point:
    def __init__(self, line) -> None:
        x, m, a, s = line.replace('{', '').replace('}', '').split(',')
        self.x = int(x[2:])
        self.m = int(m[2:])
        self.a = int(a[2:])
        self.s = int(s[2:])
    
    def total_rating(self):
        return self.x + self.m + self.a + self.s

instructions = {}
parts = []

with open("./data/day19.txt") as f:
    lines = f.read().split('\n\n')
    for line in lines[0].split('\n'):
        line = line.replace('}', '').split('{')
        instructions[line[0]] = line[1].split(',')
    for line in lines[1].split('\n'):
        if line != '':
            parts.append(Point(line))

In [2]:
len(instructions), len(parts)

(512, 200)

In [3]:
for i, (k, v) in enumerate(instructions.items()):
    if i > 5:
        break
    print(k, v)
print()
print('in', instructions['in'])

hlv ['x>1142:fgz', 'snf']
cjb ['s>2194:jl', 'mrj']
hzg ['m>2989:A', 'A']
grq ['m<453:A', 'x<1195:A', 'x>1305:R', 'R']
pm ['m>3598:mlk', 'qmh']
jsf ['s>534:R', 's<339:bf', 'a<1455:R', 'tm']

in ['a>2078:hbd', 'dn']


In [4]:
def handle_instruction(instruction, part):
    for inst in instruction:
        if ':' not in inst:
            return inst
        ind = inst.index(':')
        label = inst[0]
        operator = inst[1]
        value = int(inst[2:ind])
        goal = inst[ind+1:]

        if operator == '>':
            if getattr(part, label) > value:
                return goal
        else:
            if getattr(part, label) < value:
                return goal

In [5]:
res = 0
for part in parts:
    label = 'in'
    while label not in ['A', 'R']:
        label = handle_instruction(instructions[label], part)
    if label == 'A':
        res += part.total_rating()
res

456651

In [6]:
class PointRange:
    def __init__(self, x_min, x_max, m_min, m_max, a_min, a_max, s_min, s_max) -> None:
        self.x_min = x_min
        self.x_max = x_max
        self.m_min = m_min
        self.m_max = m_max
        self.a_min = a_min
        self.a_max = a_max
        self.s_min = s_min
        self.s_max = s_max

    def __str__(self) -> str:
        return f'{self.x_min}, {self.x_max}, {self.m_min}, {self.m_max}, {self.a_min}, {self.a_max}, {self.s_min}, {self.s_max}'
    
    def copy(self):
        return PointRange(self.x_min, self.x_max, self.m_min, self.m_max, self.a_min, self.a_max, self.s_min, self.s_max)
    
    def combinations(self):
        return (1-self.x_min+self.x_max)*(1-self.m_min+self.m_max)*(1-self.a_min+self.a_max)*(1-self.s_min+self.s_max)

In [7]:
def handle_instruction_range(instruction, part_range):
    curr_range = part_range
    res = []
    for inst in instruction:
        if ':' not in inst:
            res.append((curr_range, inst))
            break
        ind = inst.index(':')
        label = inst[0]
        operator = inst[1]
        value = int(inst[2:ind])
        goal = inst[ind+1:]

        attr_min, attr_max = getattr(curr_range, label+'_min'), getattr(curr_range, label+'_max')
        if (operator == '<' and value <= attr_min) or (operator == '>' and value >= attr_max):
            continue
        if (operator == '<' and value > attr_max) or (operator == '>' and value < attr_min):
            res.append((curr_range, goal))
            break
        if operator == '<':
            below_range = curr_range.copy()
            setattr(below_range, label+'_max', value-1)
            res.append((below_range, goal))
            setattr(curr_range, label+'_min', value)
        else:
            above_range = curr_range.copy()
            setattr(above_range, label+'_min', value+1)
            res.append((above_range, goal))
            setattr(curr_range, label+'_max', value)
    return res

In [8]:
res = []
r = PointRange(1, 4000, 1, 4000, 1, 4000, 1, 4000)
visited = set()
q = [(r, 'in')]
while q:
    r, label = q.pop()
    if (str(r), label) in visited:
        continue
    visited.add((str(r), label))
    if label == 'R':
        continue
    if label == 'A':
        res.append(r)
        continue
    q += handle_instruction_range(instructions[label], r)

In [9]:
m = 0
for r in res:
    m += r.combinations()
m

131899818301477